In [1]:
if __package__ is None:
    import sys
    from os import path
    sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))

from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import time
from datetime import datetime

In [2]:
class testcontroller1(eFMU):
    def __init__(self):
        self.input = {'a':None,'b':None}
        self.output = {'c':None}
    def compute(self):
        self.output['c'] = self.input['a'] * self.input['b']
        
class testcontroller2(eFMU):
    def __init__(self):
        self.input = {'a':None,'b':None}
        self.output = {'c':None}
        self.init = True
    def compute(self):
        print ('***Init2', self.init)
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(0.2)
        
class testcontroller3(eFMU):
    def __init__(self):
        self.input = {'a':None,'b':None}
        self.output = {'c':None}
        self.init = True
    def compute(self):
        print ('***Init3', self.init)
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(0.2)

In [11]:
from datetime import datetime
import pandas as pd
controller = {}
controller['forecast1'] = {'fun': testcontroller1, 'sampletime': 3}
controller = controller_stack(controller, tz=-8, debug=True, parallel=True)
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
controller.initialize(mapping)
now = time.time()
print(datetime.now())
while time.time() - now < 16:
    controller.query_control(time.time())
print(pd.DataFrame(controller.log_to_df()['forecast1']))


2020-08-25 17:12:40.808267
                                  a    b     c     Logging
2020-08-25 15:49:54.680294991   NaN  NaN   NaN  Initialize
2020-08-25 16:12:40.808634043  10.0  4.0  40.0        None
2020-08-25 16:12:43.808945894  10.0  4.0  40.0        None
2020-08-25 16:12:46.809303999  10.0  4.0  40.0        None
2020-08-25 16:12:49.809392214  10.0  4.0  40.0        None
2020-08-25 16:12:52.810004950  10.0  4.0  40.0        None
2020-08-25 16:12:55.810565948  10.0  4.0  40.0        None


In [ ]:
controller = {}
controller['forecast1'] = {'fun':testcontroller2, 'sampletime':1}
controller['mpc1'] = {'fun':testcontroller3, 'sampletime':'forecast1'}
controller['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controller['forecast2'] = {'fun':testcontroller2, 'sampletime':2}
controller['forecast3'] = {'fun':testcontroller1, 'sampletime':2}

# Initialize controller
controller = controller_stack(controller, tz=-8, debug=True, parallel=True)
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

controller.initialize(mapping)

In [ ]:
controller.query_control(time.time())

In [ ]:
controller.query_control(time.time())

In [ ]:
controller.query_control(time.time())


In [ ]:
controller.query_control(time.time())


In [ ]:
controller.log_to_df()